In [1]:
import pandas as pd
tr = pd.read_csv('tr.csv')
te = pd.read_csv('te.csv')
y = pd.read_csv('y_target.csv')
test = pd.read_csv('test.csv')
test_id = test['ID']

In [2]:
y=y['total_rental']

# GradientBoostingRegressor

## grid search_1트

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(tr,y,shuffle=True,random_state=42)

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

gb = GradientBoostingRegressor()


param_grid = {'learning_rate': [0.01,0.03,0.05],
                  'subsample'    : [0.9, 0.5, 0.2],
                  'n_estimators' : [100,500,1000],
                  'max_depth'    : [4,6,8]
                 }

gb_grid = GridSearchCV( estimator=gb, param_grid=param_grid,  n_jobs=4, cv=5) 

gb_grid.fit(x_train,y_train)
print('final params', gb_grid.best_params_)  
print('best score', gb_grid.best_score_) 

final params {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 1000, 'subsample': 0.2}
best score 0.9290568060085


## 결과

In [3]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(tr,y,shuffle=True,random_state=42)
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(learning_rate= 0.01, max_depth= 6, n_estimators= 1000, subsample= 0.2)
gb.fit(x_train,y_train)
y_pred_gb = gb.predict(x_test)
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(y_test, y_pred_gb, squared=False)
rmse

505.06041469547193

## grid search_2트

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(tr,y,shuffle=True,random_state=42)

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

gb = GradientBoostingRegressor()


param_grid = {'learning_rate': [0.001,0.005,0.01],
                  'subsample'    : [0.5, 0.2,0.1],
                  'n_estimators' : [800,1000,1500],
                  'max_depth'    : [5,6,7]
                 }

gb_grid = GridSearchCV( estimator=gb, param_grid=param_grid,  n_jobs=4, cv=5) 

gb_grid.fit(x_train,y_train)
print('final params', gb_grid.best_params_)  
print('best score', gb_grid.best_score_) 

final params {'learning_rate': 0.005, 'max_depth': 7, 'n_estimators': 1500, 'subsample': 0.2}
best score 0.9296549449620596


## 결과

In [3]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(tr,y,shuffle=True,random_state=42)
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(learning_rate= 0.005, max_depth= 7, n_estimators= 1500, subsample= 0.2)
gb.fit(x_train,y_train)
y_pred_gb = gb.predict(x_test)
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(y_test, y_pred_gb, squared=False)
rmse

499.79265320931

# submission 제출

In [4]:
final = gb.predict(te)
sub = pd.DataFrame(columns=['ID','total_rental'])
sub['ID']=test_id
sub['total_rental'] = final

In [5]:
result = sub.to_csv('grid_gb_2.csv',index=False)
a=pd.read_csv('grid_gb_2.csv')
a.head()

,ID,total_rental
0,A600,7163.418497
1,A601,7038.196084
2,A602,7082.885251
3,A603,5771.620019
4,A604,5160.387437


# train 전체 적용 시킨 모델

In [3]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(tr,y,shuffle=True,random_state=42)
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(learning_rate= 0.005, max_depth= 7, n_estimators= 1500, subsample= 0.2)
gb.fit(tr,y)
final = gb.predict(te)
sub = pd.DataFrame(columns=['ID','total_rental'])
sub['ID']=test_id
sub['total_rental'] = final
result = sub.to_csv('grid_gb_3.csv',index=False)
a=pd.read_csv('grid_gb_3.csv')
a.head()

,ID,total_rental
0,A600,7200.251229
1,A601,7078.774184
2,A602,7429.761699
3,A603,6066.200327
4,A604,4984.310139


# voting

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(tr,y,shuffle=True,random_state=42)
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(bootstrap= True, max_depth= 10, max_features= 'auto',
                           min_samples_leaf= 1, min_samples_split= 2,n_estimators= 200)

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(tr,y,shuffle=True,random_state=42)
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(learning_rate= 0.005, max_depth= 7, n_estimators= 1500, subsample= 0.2)

In [8]:
from sklearn.ensemble import VotingRegressor

voting_model = VotingRegressor([('rf', rf), ('gb', gb)])

voting_model.fit(tr , y)

final = voting_model.predict(te)

In [9]:
sub = pd.DataFrame(columns=['ID','total_rental'])
sub['ID']=test_id
sub['total_rental'] = final

In [10]:
result = sub.to_csv('voting.csv',index=False)
a=pd.read_csv('voting.csv')
a.head()

,ID,total_rental
0,A600,7139.699681
1,A601,7067.940568
2,A602,7335.380098
3,A603,5844.411310
4,A604,5033.936906
